In [1]:
import glob
import pickle
from pathlib import Path

import numpy as np
import torch

from torch.utils.data import Dataset, IterableDataset, DataLoader

In [3]:
class PklDataset(Dataset):
    """Get a pkled file from the folder and return
    the formatted dataset that corresponds to it"""

    # pylint: disable=no-member
    # pylint: disable=too-many-locals
    def __init__(self, root_dir):
        super().__init__()
        """
        Args:
            root_dir (string): Directory with all the data.
        """

        self.root_dir = root_dir
        self.categories = [
            "arccos",
            "arccosh",
            "arcsin",
            "arcsinh",
            "arctan",
            "arctan2",
            "arctanh",
            "heaviside",
            "log",
            "log10",
            "log1p",
            "log2",
            "multiply",
            "sin",
            "sinh",
            "sqrt",
            "square",
            "tan",
            "tanh",
            "divide",
            "add",
            "subtract",
            "linear",
            "exponential",
            "cubic",
            "joint_normal",
            "step",
            "quadratic",
            "w_shaped",
            "spiral",
            "logarithmic",
            "fourth_root",
            "sin_four_pi",
            "sin_sixteen_pi",
            "two_parabolas",
            "circle",
            "ellipse",
            "diamond",
            "multiplicative_noise",
            "multimodal_independence",
        ]

    def __len__(self):
        """return the total number of files in the directory."""
        return len(glob.glob(str(Path(self.root_dir, "*.pkl"))))
    
    def __getitem__(self, idx, img_size=128):
        """ get the item corresponding to the index and pad the samples
        to have img_size in both dimensions. Returns a tensor with all
        indices concatenated"""

        if torch.is_tensor(idx):
            idx = idx.tolist()

        final_maps_tensor = torch.empty(size=(0, 1, img_size, img_size))
        final_labels_tensor = torch.empty(size=(0, len(self.categories)))

       
        with open(Path(self.root_dir, str(idx) + ".pkl"), "rb") as file_path:
            data_dictionary = pickle.load(file_path)

        temp_map_tensor = torch.empty(
            size=(len(data_dictionary), 1, img_size, img_size)
        )
        temp_labels_tensor = torch.empty(
            size=(len(data_dictionary), len(self.categories))
        )

        for i, (_, entry) in enumerate(data_dictionary.items()):
            mgc_map, _, labels = entry

            # pad the datasets with wrong size
            if any(np.array(mgc_map.shape) != img_size):
                num_missing = np.ones(2) * img_size - np.array(mgc_map.shape)

                # if not even add one pad in that dimension,
                # and then progress as if the number of
                #  missing was even
                even = np.array(num_missing % 2 == 0)
                if not all(even):
                    # get the numerical values
                    odd = (~even).astype(int)
                    padding_mask = ((0, odd[0]), (0, odd[1]))
                    # pad 1 time
                    mgc_map = np.pad(mgc_map, pad_width=padding_mask, mode="edge")
                    # update the number of missing
                    num_missing = num_missing - even

                num_missing = (num_missing / 2).astype(int)
                padding_mask = ((num_missing[0],), (num_missing[1],))
                mgc_map = np.pad(mgc_map, pad_width=padding_mask, mode="edge")

            # set the temporary tensors
            temp_map_tensor[i, 0, :, :] = (
                torch.from_numpy(mgc_map).double().unsqueeze(0)
            )
            # get the numerical labels
            # this works for BCELoss
            temp_labels_tensor[i, :] = torch.from_numpy(
                np.isin(self.categories, labels)
            ).double()

        final_maps_tensor = torch.cat((final_maps_tensor, temp_map_tensor), dim=0)
        final_labels_tensor = torch.cat(
            (final_labels_tensor, temp_labels_tensor), dim=0
        )

        return final_maps_tensor, final_labels_tensor
    @staticmethod
    def collate_fn(samples):
        """
        redefine the collate_fn to be able to stack the arrays that are
        randomly sized in the batch dimension
        """

        # get the sequences
        maps_tensors, labels_tensors = zip(*samples)

        # concatenate them
        maps_tensors_concatenated = torch.cat(maps_tensors, dim=0)
        labels_tensors_concatenated = torch.cat(labels_tensors, dim=0)

        return maps_tensors_concatenated, labels_tensors_concatenated




root_path = Path('C:\machine_learning\MGC_classifier\data')
test_dataset = PklDataset(root_path)
test_dlr = DataLoader(
    dataset = test_dataset, 
    collate_fn=PklDataset.collate_fn,
    batch_size=3
)
for i, test_sample in enumerate(test_dlr):
    print(test_sample[0].shape, test_sample[1].shape)
    if i>3:
        break


torch.Size([242, 1, 128, 128]) torch.Size([242, 40])
torch.Size([242, 1, 128, 128]) torch.Size([242, 40])
torch.Size([251, 1, 128, 128]) torch.Size([251, 40])
torch.Size([246, 1, 128, 128]) torch.Size([246, 40])
torch.Size([244, 1, 128, 128]) torch.Size([244, 40])


# minibatch dataset

In [4]:
class MinibatchDataset(Dataset):
    """Get a set of tensors and target classes
    a batch and return a minibatch for that batch"""

    def __init__(self, data):
        super().__init__()

        self.data = data

    def __len__(self):
        """return the total number of samples in the batch."""
        return self.data[1].shape[0]

    def __getitem__(self, idx):
        """ get the correct indices"""
        inputs, targets = self.data[0][idx, :, :, :], self.data[1][idx, :]
        return inputs, targets
    

330

In [5]:
minibatch_ds = MinibatchDataset(test_sample)
dl = DataLoader(minibatch_ds, batch_size=2, shuffle=True,)
for sample in dl:
    print(sample[0].shape, sample[1].shape)

torch.Size([2, 1, 128, 128]) torch.Size([2, 40])
torch.Size([2, 1, 128, 128]) torch.Size([2, 40])
torch.Size([2, 1, 128, 128]) torch.Size([2, 40])
torch.Size([2, 1, 128, 128]) torch.Size([2, 40])
torch.Size([2, 1, 128, 128]) torch.Size([2, 40])
torch.Size([2, 1, 128, 128]) torch.Size([2, 40])
torch.Size([2, 1, 128, 128]) torch.Size([2, 40])
torch.Size([2, 1, 128, 128]) torch.Size([2, 40])
torch.Size([2, 1, 128, 128]) torch.Size([2, 40])
torch.Size([2, 1, 128, 128]) torch.Size([2, 40])
torch.Size([2, 1, 128, 128]) torch.Size([2, 40])
torch.Size([2, 1, 128, 128]) torch.Size([2, 40])
torch.Size([2, 1, 128, 128]) torch.Size([2, 40])
torch.Size([2, 1, 128, 128]) torch.Size([2, 40])
torch.Size([2, 1, 128, 128]) torch.Size([2, 40])
torch.Size([2, 1, 128, 128]) torch.Size([2, 40])
torch.Size([2, 1, 128, 128]) torch.Size([2, 40])
torch.Size([2, 1, 128, 128]) torch.Size([2, 40])
torch.Size([2, 1, 128, 128]) torch.Size([2, 40])
torch.Size([2, 1, 128, 128]) torch.Size([2, 40])
torch.Size([2, 1, 12

# future to do
Implement iterable dataset to be able to continuously get data